In [1]:
from jax_md import *
import jax
import jax.random as jr
import jax.numpy as jnp
import matplotlib.pyplot as plt

# jax.config.update("jax_enable_x64", True)

setup

In [2]:
dim = 1
N = 512
box_size = 1

displacement stuff

In [3]:
displacement, shift = space.periodic(box_size)
metric = space.metric(displacement)
v_displacement = space.map_product(displacement)
v_metric = space.map_product(metric)

brownian simulation

In [10]:
def brownian_simulation(key, temperature, dt, steps, energy_fn):
    pos_key, sim_key, times_key = jr.split(key, 3)
    # R = jr.uniform(pos_key, (N,1), maxval=box_size)
    R = 0.1*jr.normal(pos_key, shape=(N,1 ))+0.5

    init_fn, apply_fn = simulate.brownian(energy_fn, shift, dt, temperature)
    initial_state = init_fn(sim_key, R)
    state = init_fn(sim_key, R)

    do_step = lambda state, t: (apply_fn(state), apply_fn(state))
    final_state, intermediate_states = jax.lax.scan(do_step, state, steps)

    return initial_state.position, final_state.position, intermediate_states.position, times_key

In [11]:
initial, final, intermediate, times_key = brownian_simulation(jr.PRNGKey(0), 0.01, 0.01, jnp.arange(10000), lambda r: 0.)

In [12]:
final = jnp.expand_dims(final, 0)
initial = jnp.expand_dims(initial, 0)
all_states = jnp.concatenate((initial, intermediate, final), axis=0)
print(jnp.shape(all_states))

(10002, 512, 1)


In [13]:
jnp.save("test", all_states)